<a href="https://colab.research.google.com/github/gremlin97/EVA-8/blob/main/LLM-Experiments/RAG_Orca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output

In [ ]:
# # Install required packages
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install pypdf
!pip install chromadb
clear_output()

# Retrieval

In [ ]:
!pip install sentence_transformers > /dev/null

In [ ]:
from IPython.display import clear_output, Image
import os
import torch
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import LlamaCppEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [ ]:
%mkdir pdfs
%cd pdfs
!wget "https://www.dropbox.com/scl/fi/t6douu3clyj1ounm63owm/Bro-et-al.-2019-JGR-P-Subsurface-Sediment-Mobilization-in-the-Southern-Part-of-Chryse-Planitia-Mars.pdf?dl=0&rlkey=lvbsenx6etorwotqxoe9ozijz" -O data.pdf
!wget "https://esurf.copernicus.org/articles/11/633/2023/esurf-11-633-2023.pdf" -O data2.pdf
%cd ..
clear_output()

In [ ]:
# embed = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
# embed = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
encode_kwargs={'normalize_embeddings': True}
embed = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5",model_kwargs={'device':'cuda'},encode_kwargs=encode_kwargs)
clear_output()

In [ ]:
# %%time
# # Directory containing the PDF files
# pdf_directory = 'pdfs'

# # Get the list of files in the current directory
# filenames = os.listdir(pdf_directory)

# # Initialize an empty list to store all_splits
# all_splits = []

# # Loop through all PDF files in the directory
# for filename in filenames:
#     if filename.endswith('.pdf'):
#         # Load and split the current PDF file
#         loader = PyPDFLoader(os.path.join(pdf_directory, filename))
#         pages = loader.load_and_split()

#         # Split text into chunks of size 1000 with a chunk overlap of 200 to maintain some flow
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#         splits = text_splitter.split_documents(pages)

#         # Append the splits of the current PDF to the all_splits list
#         all_splits.extend(splits)

# print(len(all_splits))

# # Q and A with vector store, retrieve relevant context/embeddings, and answer
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed)

335
CPU times: user 32.1 s, sys: 844 ms, total: 33 s
Wall time: 38.7 s


In [ ]:
# %%time
# query = "What are the features of Mars Cones?"
# docs = vectorstore.similarity_search(query)

CPU times: user 54.7 ms, sys: 0 ns, total: 54.7 ms
Wall time: 107 ms


In [ ]:
# for x in docs:
#   print(x.page_content)
#   print("\n")

et al., 2021; Ye et al., 2021; Zhao et al., 2021). In the ﬁeld,
there is one pitted cone with a height of 80 m and a basal di-
ameter of 800 m located about 16 km south-east of the land-
ing site, and it has attracted the attention of the mission sci-
ence team (Liu et al., 2021). Alternative interpretations, in-
cluding cinder cones, sedimentary volcanoes, or pingos, have
been proposed to explain the origin of this structure. Sedi-
mentary volcanism appears to be the preferred origin as re-
ported in recent studies (Ye et al., 2021; Huang et al., 2022),
although other small mounds in the region have been inter-
preted as lava domes (Lin et al., 2023). In situ study of the
closest cone to the landing site would provide a great oppor-
tunity for ground-truthing one example of putative sedimen-
tary volcanoes on Mars. The identiﬁcation of clay minerals,
like smectite or illite (Mazzini and Etiope, 2017, and refer-


the formation of speci ﬁc surface landforms, such as lineated depression

# RAG

In [ ]:
!pip install -q transformers
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir
!pip install -q llama-index
clear_output()

In [ ]:
import torch
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

Exception ignored in: <function _xla_gc_callback at 0x78a341500550>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


total size (MB): 4368.44


4167it [00:38, 108.20it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
documents = SimpleDirectoryReader("/content/pdfs").load_data()

In [ ]:
len(documents)

47

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Extract comprehensive information regarding the characteristics and distinctive features of the geological feature “cone”, with a particular focus on cones found on planetary bodies such as Mars. This information should encompass details about the physical attributes, dimensions, and visual appearance of cones. Additionally, please investigate any variations in cone attributes based on different geological regions or locations. The aim is to gain an in-depth understanding of the appearance and characteristics of cones in the context of planetary geology.")

In [ ]:
print(response)

 Cones are geological features that form as a result of volcanic activity, erosion, or other geological processes. They are characterized by their steep slopes and rounded summit areas, with small knobs or bumps on the surface. The size and shape of cones can vary greatly depending on the specific process that formed them and the geological context in which they are found.

On Mars, cones are commonly found in association with volcanic craters and lava flows. They are often characterized by their relatively flat summit areas and the presence of small knobs or bumps on the surface. The circularity of these cones, their gradual transition into the surrounding plains, and the absence of fracture patterns such as radial faults suggest that they were formed by the ascent of material from the subsurface.

In addition to their physical attributes, cones on Mars can also be distinguished by their visual appearance. Some cones have reddish-brown colors due to the presence of iron oxide (rust) o

In [ ]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response)

Does this document say anything about cones on Mars?


Llama.generate: prefix-match hit


 Yes, this document mentions cones on Mars. On page 650, it states that an in situ study of the closest cone to the landing site would provide a great opportunity for ground-truthing one example of putative sedimentary volcanoes on Mars. Additionally, on page 654, it describes observations of pitted cones and domes on Mars using MOC, THEMIS, and TES data.
Tell me all the content this document has about the features of Mars Cones


Llama.generate: prefix-match hit


 The document "data2.pdf" contains two references related to the features of Mars cones. The first reference is a research paper by Farrand, Gaddis, and Keszthlyi (2005) titled "Pitted cones and domes on Mars: Observations in Acidalia Planitia and Cydonia Mensae using MOC, THEMIS, and TES data" which was published in the Journal of Geophysical Research. The second reference is an in situ study by Mazzini and Etiope (2017) titled "Identification of clay minerals on Mars: A case study at the landing site of the Mars Science Laboratory rover" which was published in Earth Surface Dynamics. Both references discuss the features of Mars cones, including their morphology, composition, and potential as sedimentary volcanoes.
What are the features of cones on Mars?


Llama.generate: prefix-match hit


 Based on the given context information, cones on Mars can have features such as pitted surfaces, which may be indicative of a mud volcano origin. Additionally, the presence of clay minerals like smectite or illite could also provide evidence for sedimentary volcanism on Mars.
What model are you?


Llama.generate: prefix-match hit


 Based on the given context information, it appears that you are an AI assistant designed to help with research related to geology and planetary science. You have access to data related to the study of edi (extraterrestrial impact craters) in a specific area, including measurements of their height and classification based on various morphological features. Additionally, you have information about the age determination of these craters using crater model ages.


KeyboardInterrupt: ignored

In [ ]:
response = query_engine.query("<summarization_query>")

# Testing

In [ ]:
# from langchain.prompts import PromptTemplate

# template = """Use the following pieces of context to answer the question at the end.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# {context}
# Question: {question}
# Helpful Answer:"""

# prompt = PromptTemplate(
#     template=template, input_variables=["context", "question"]
# )

# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())

# question = "What is this pdf about?"
# result = qa_chain({"query": question})
# result["result"]
